In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
!pip install nlpaug
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 1.9 MB 12.1 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 3.3 MB 55.3 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=c33c3de836566914705765b5a380b27718f5268db470a657240caaac98f6760d
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 2.6 MB 15.4 MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-py3-none-any.whl size=7053 sha256=d1465dd862ba23498080451eff9aa2b676051d53493845432274c944fa04eb54
  Stored in directory: /root/.cache/pip/wheels/12/40/73/a0765d65e793332b79dfe6c34c713e7c0066ea785191b3f50a
Successfully built language-detector
     |████████████████████████████████| 1.2 MB 14.8 MB/s 
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=94fada4c9a862a0d

True

In [4]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [12]:
from db.db import DB, AugmentedDB

dataset_type = "wiki"

In [ ]:
db = DB(dataset_type)
augmented_db = AugmentedDB(dataset_type)

In [ ]:
def get_data(db, pct=0.1, max_seg_size=256):
    segments = db.get_random_segments_pct(pct_data=pct, max_segment_size=max_seg_size)
    
    return segments

In [ ]:
regular_segments = db.get_random_segments_pct(pct_data=0.1, max_segment_size=10)
augmented_segments = augmented_db.get_random_segments_pct(pct_data=0.1, max_segment_size=10)

In [ ]:
len(regular_segments), len(augmented_segments)

(213, 0)

## Tokenizing

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
encoding = tokenizer.encode_plus(text,
                                 add_special_tokens = True,
                                 truncation = True,
                                 padding = "max_length",
                                 return_attention_mask = True,
                                 return_tensors = "pt")

NameError: ignored

## GPT Augmentor

In [6]:
from src.dataset.gpt_augmentor import Augmentor

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
result = Augmentor.augment_gpt2(["I enjoy walking with my cute dog", "I enjoy walking with my cute dog"], fast=True)

In [ ]:
result

[["I enjoy walking with my cute dog. It's so good to have the chance to walk with a dog. But I have not been able to find a puppy that I would want to adopt or get the chance to look for.\n\nI am",
  "I enjoy walking with my cute dog, she loves taking trips to different places on the planet, even in the desert! The world seems so beautiful and beautiful to me - it is so beautiful when my dogs go to the desert and it's like it",
  'I enjoy walking with my cute dog and playing with our kids," said David J. Smith, director of the Humane Society of the US.\n\n"So as a result, I\'ve got more work in my time," he said.\n\n'],
 ["I enjoy walking with my cute dog. It's so good to have the chance to walk with a dog. But I have not been able to find a puppy that I would want to adopt or get the chance to look for.\n\nI am",
  "I enjoy walking with my cute dog, she loves taking trips to different places on the planet, even in the desert! The world seems so beautiful and beautiful to me - it is so

## Insert GPT Augmented Data

In [7]:
from db.db import DB, AugmentedDB, ReverseAugmentedDB
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
dataset_type = "wiki"

In [ ]:
db = DB(dataset_type)
augmented_db = AugmentedDB(dataset_type)
reverse_augmented_db = ReverseAugmentedDB(dataset_type)

In [ ]:
min_sent_tokens = 8
max_sent_tokens = 64

In [ ]:
target_sentences_original = db.get_target_sentences()
target_sentences = [s[1] for s in target_sentences_original]
cleaned_target_sentences = []

In [ ]:
for s in target_sentences:
    if len(word_tokenize(s)) > min_sent_tokens:
        shortened_sentence = " ".join(word_tokenize(s)[:max_sent_tokens])
        cleaned_target_sentences.append(shortened_sentence)

In [ ]:
augmented_segments = Augmentor.augment_gpt2(cleaned_target_sentences[:2], 
                                            fast=True, 
                                            # multiply by 5 to account for 5 as a max segment
                                            max_seq_word_length=max_sent_tokens*5, 
                                            verbose=True)

Completed augmenting 1/2...
Completed augmenting 2/2...


In [ ]:
# # TESTING

# # reverse augmented db
# for options in augmented_segments:
#     # reverse the order of the sentences making the last one in the order the new target
#     # sentence for this particular segment. Leave the original target sentence out
#     # to avoid confusion in the model (go up to -1)
#     for option in options:
#         # print(option)
#         segment = nltk.tokenize.sent_tokenize(option)
#         print(segment)
#         print(segment[::-1])
        
#         # print(segment)

In [ ]:
# augmented db
for options in augmented_segments:
    for option in options:
        segment = nltk.tokenize.sent_tokenize(option)
        
        augmented_db.create_segment(segment)

In [ ]:
# reverse augmented db
for options in augmented_segments:
    # reverse the order of the sentences making the last one in the order the new target
    # sentence for this particular segment. Leave the original target sentence out
    # to avoid confusion in the model (go up to -1)
    for option in options:
        segment = nltk.tokenize.sent_tokenize(option)
        
        reverse_augmented_db.create_segment(segment[::-1])

## Batch Augmentation

In [ ]:
min_sent_tokens = 8
max_sent_tokens = 64

for dataset_type in ["wiki", "clinical", "fiction"]:
  db = DB(dataset_type)
  augmented_db = AugmentedDB(dataset_type)
  reverse_augmented_db = ReverseAugmentedDB(dataset_type)

  target_sentences_original = db.get_target_sentences()
  target_sentences = [s[1] for s in target_sentences_original]
  cleaned_target_sentences = []

  # clean sentences
  for s in target_sentences:
    if len(word_tokenize(s)) > min_sent_tokens:
        shortened_sentence = " ".join(word_tokenize(s)[:max_sent_tokens])
        cleaned_target_sentences.append(shortened_sentence)

  # do 1 at a time
  for target_sentence in cleaned_target_sentences[100:500]:
    augmented_segments = Augmentor.augment_gpt2([target_sentence], 
                                                fast=True, 
                                                # multiply by 5 to account for 5 as a max segment
                                                max_seq_word_length=max_sent_tokens*5, 
                                                verbose=True)
    
    # reverse augmented db
    for options in augmented_segments:
        # reverse the order of the sentences making the last one in the order the new target
        # sentence for this particular segment. Leave the original target sentence out
        # to avoid confusion in the model (go up to -1)
        for option in options:
            segment = nltk.tokenize.sent_tokenize(option)
            print(segment)
            
            try:
              # reverse the segment and go until the original target sentence, but exclude it.
              reverse_augmented_db.create_segment(segment[::-1][:-1])
            except Error as e:
              print("something went wrong", e)

## Dataset Testing

In [ ]:
from src.dataset.albert import AlbertDataset

In [ ]:
dataset.num_samples

795

In [ ]:
sentences, tokenized_sentences, labels = dataset.process()

In [ ]:
len(sentences)

795

### Experiment Testing

In [ ]:
import copy

config_overrides = {
        "bert_type": ["ldabert"],
        "dataset_type": ["clinical"],
        "final_dropout": [0.5, 0.8],
        "dense_neurons": [64, 128, 256],
        "pct_data": [1],
        "augment_pct": [0.1, 0.25, 0.5, 1],
        "epochs": [1000]
    }

experiments = [{},]

for k,v in config_overrides.items():
    new_values = len(v)
    current_exp_len = len(experiments)
    for _ in range(new_values-1):
       experiments.extend(copy.deepcopy(experiments[:current_exp_len]))
    for validx in range(len(v)):
       for exp in experiments[validx*current_exp_len:(validx+1)*current_exp_len]:
            exp[k] = v[validx]

print(experiments)

[{'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.5, 'dense_neurons': 64, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 1000}, {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.8, 'dense_neurons': 64, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 1000}, {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.5, 'dense_neurons': 128, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 1000}, {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.8, 'dense_neurons': 128, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 1000}, {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.5, 'dense_neurons': 256, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 1000}, {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.8, 'dense_neurons': 256, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 1000}, {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.5, 'dense_neurons': 64, 'pct_data':

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(experiments)

,bert_type,dataset_type,final_dropout,dense_neurons,pct_data,augment_pct,epochs
0,ldabert,clinical,0.5,64,1,0.10,1000
1,ldabert,clinical,0.8,64,1,0.10,1000
2,ldabert,clinical,0.5,128,1,0.10,1000
3,ldabert,clinical,0.8,128,1,0.10,1000
4,ldabert,clinical,0.5,256,1,0.10,1000
5,ldabert,clinical,0.8,256,1,0.10,1000
6,ldabert,clinical,0.5,64,1,0.25,1000
7,ldabert,clinical,0.8,64,1,0.25,1000
8,ldabert,clinical,0.5,128,1,0.25,1000
9,ldabert,clinical,0.8,128,1,0.25,1000
